In [2]:
# load python modules
from Bio import Entrez
from collections import OrderedDict
import pandas as pd

In [3]:
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows', None)

In [4]:
#set your file pathway for saving both output files
GeneOutputFile = './nearestGenes.csv' #this is default. Change to something else if you like
SNPInfoOutputFile = './SNPInfo.csv' #default or leave blank if you don't want to store this info


In [5]:
# initialize some default parameters
Entrez.email = 'your@email.com'  # provide your email address
db = 'snp'                       # set search to snp database
paramEutils = { 'usehistory':'Y',  
              "retmode":'xml',"retmax": 10} #Use Entrez search history to cache results

In [6]:
def get_idList(snp_id):
    #data must be in xml format
    eSearch = Entrez.esearch(db=db, term=snp_id, **paramEutils)
    res = Entrez.read(eSearch)
    idList = res['IdList'] #returns a list of id's to create summaries for
    return(idList)

In [7]:
 def parse_return_xml(dict_obj):
    '''Returns a dictionary of keys from entrez summary result using 'snp' database
    Entrez.esummary(db=db, id='rs####', retmax=retmax, retmode=retmode)'''
    
    record = {}
    # info to collect is three layers in
    first_order = dict_obj['DocumentSummarySet']
    second_order = first_order['DocumentSummary'] 
    
    #return the dictionary of summary
    record = second_order[0]
    
    return(record)

In [21]:
def get_closest_genes(list_of_snps):
    '''Calls ncbi snp database with an rs## snp id from a input list, and returns 2 dataframes
    with a row for each snp id: snp_records df contains comprehensive snp data and genes df 
    contains info for genes associated with each snp
    
    snp_record df schema: 
    ACC : string                                
    ALLELE: Bool 
    ALLELE_ORIGIN: string : rs###, however most of the time is blank 
    CHR : string chromosome snp found on 
    CHRPOS : string chr:bp
    CHRPOS_PREV_ASSM : string chr:bp from previous assembly
    CHRPOS_SORT : string chromosome position but with 000 in front
    CITED_SORT : string string
    CLINICAL_SIGNIFICANCE : string
    CLINICAL_SORT : string 
    CREATEDATE : data record of snp was created
    DOCSUM : string summary of all columns
    FXN_CLASS : string comma separate (i.e. 'intron_variant,genic_downstream_transcript_variant')
    GENES : list of dictionaries of genes (i.e. [{'NAME': 'LRRC8D', 'GENE_ID': '55144'}])
    GLOBAL_MAFS : list of dictionary (i.e. {'STUDY': '1000Genomes', 'FREQ': 'C=0.049521/.....)
    GLOBAL_POPULATION : string
    GLOBAL_SAMPLESIZE : string
    HANDLE : string reported biobanks with snp? (i.e. 1000GENOMES,EVA_UK10K_TWINSUK,....)
    IDList : list of uid's generated from first get_idList(snp)
    MERGED_SORT : string
    ORIG_BUILD : string
    SNP_CLASS : string (i.e. 'snv')
    SNP_ID : string
    SNP_ID_SORT : string
    SPDI : string (i.e. 'NC_000005.10:95305173:T:C')
    SS : string (i.e. '10263196,82343251,82652005,112205736,165508732')
    SUSPECTED : string
    TAX_ID : string
    TEXT : string (i.e. 'MergedRs=6896334')
    UPDATEDATE : Date
    UPD_BUILD : string
    VALIDATED : string (i.e. 'by-frequency,by-alfa,by-cluster')
    snpID : string rs###
    
    gene_records schema:
    GENE_ID : int ID#
    NAME : string common name 
    ACC : string NC_000... 
    snpID : string rs###
    '''
    #initialize empty dataframe with be adding onto at each iteration
    snp_records = pd.DataFrame()
    genes = pd.DataFrame()
    
    #iterated over each snp to create row for dataframe
    for snp in list_of_snps:
        #every snp may have several ids associated with it
        idList = get_idList(snp)
        if len(idList) > 0: # need an id to call summary, in rare instances snps don't have a uid
            eSummary = Entrez.esummary(db=db, id=idList[0], **paramEutils) #only need one uid so get the first occurence
            summary_dict = Entrez.read(eSummary)
            record = parse_return_xml(summary_dict)
            record['IDList'] = idList
            #Create new dataframe for one record with index as snp
            new_record = pd.DataFrame()
            new_record = new_record.append(record, ignore_index=True)
            new_record.index = [snp]
            #append to existing dataframe with all snps
            snp_records = snp_records.append(new_record)
            if len(record['GENES']) > 0: #parse genes out into separate dataframe
                new_gene = pd.DataFrame(data=record['GENES'],index=range(len(record['GENES'])))
                new_gene['snpId'] = snp
                genes = genes.append(new_gene)
                
    snp_records['snpId'] = snp_records.index
                
    return(snp_records, genes)
        

In [10]:
snp_df = pd.read_csv('pathway/to/file/with/your/snp/data')

snp_df.head() #this is an example of the file I had from epistatic analysis using mbmdr

,Unnamed: 0,first,second,chi_squared,p-value
0,0,rs6896334,rs16957704,166.321,0.001
1,1,rs187652,rs16957704,148.227,0.001
2,2,rs154058,rs16957704,144.19799999999995,0.001
3,3,rs17498135,rs6430527,137.342,0.001
4,4,rs640333,rs3011830,136.56,0.001


In [19]:
#get a list of snps from a file and read into a df above
snp_list = snp_df['column with your snps'].values #this is the list you will use for the function

#or create a list manually if you don't have a file with snps
#snp_list = [rs1, rs2, etc]

# Get gene and snp information 

In [24]:
#I suggest trying it with only 10 to see if it works (i.e. snp_list[:10])
SNPInfo, GeneInfo = get_closest_genes(snp_list)


snp =  rs640333
id list =  ['60232172', '57110850', '640333']
snp =  rs4727608
id list =  ['61297825', '61123651', '56486996', '10346690', '4727608']
snp =  rs12490758
id list =  ['61190134', '12490758']
snp =  rs934383
id list =  ['59965562', '934383']
snp =  rs7826843
id list =  ['58899764', '56626120', '7826843']


In [25]:
#Look at shape of each to ensure all of the snps ran. I ran into problems using > 100 snps at a time.
print(GeneInfo.shape) #gene df will be smaller as not all snps with have genes associated with it
print(SNPInfo.shape) 

(3, 3)
(5, 33)


# Save dataframes to file

In [26]:
GeneInfo.to_csv(GeneOutputFile,index=False)
SNPInfo.to_csv(SNPInfoOutputFile,index=False)

# GeneInfo Schema:
GENE_ID : int ID#

NAME : string common name 

ACC : string NC_000... 

index : string rs### 

# SNPInfo Schema

ACC : string                                

ALLELE: Bool 

ALLELE_ORIGIN: string : rs###, however most of the time is blank 

CHR : string chromosome snp found on 

CHRPOS : string chr:bp

CHRPOS_PREV_ASSM : string chr:bp from previous assembly

CHRPOS_SORT : string chromosome position but with 000 in front

CITED_SORT : string string

CLINICAL_SIGNIFICANCE : string

CLINICAL_SORT : string 

CREATEDATE : data record of snp was created

DOCSUM : string summary of all columns

FXN_CLASS : string comma separate (i.e. 'intron_variant,genic_downstream_transcript_variant')

GENES : list of dictionaries of genes (i.e. [{'NAME': 'LRRC8D', 'GENE_ID': '55144'}])

GLOBAL_MAFS : list of dictionary (i.e. {'STUDY': '1000Genomes', 'FREQ': 'C=0.049521/.....)

GLOBAL_POPULATION : string

GLOBAL_SAMPLESIZE : string

HANDLE : string reported biobanks with snp? (i.e. 1000GENOMES,EVA_UK10K_TWINSUK,....)

IDList : list of uid's generated from first get_idList(snp)

MERGED_SORT : string

ORIG_BUILD : string

SNP_CLASS : string (i.e. 'snv')

SNP_ID : string

SNP_ID_SORT : string

SPDI : string (i.e. 'NC_000005.10:95305173:T:C')

SS : string (i.e. '10263196,82343251,82652005,112205736,165508732')

SUSPECTED : string

TAX_ID : string

TEXT : string (i.e. 'MergedRs=6896334')

UPDATEDATE : Date

UPD_BUILD : string

VALIDATED : string (i.e. 'by-frequency,by-alfa,by-cluster')